In [31]:
from config import VK_TOKEN

import tqdm
import json
import vk_api
import re
import pathlib

In [32]:
vk_session = vk_api.VkApi(token=VK_TOKEN, app_id=5820017)
vk_session._auth_token()

vk = vk_session.get_api()

In [33]:
tools = vk_api.VkTools(vk_session)

In [34]:
recipes_publics = [-35838585, -43818640, -27740785]

In [35]:
a="""
https://vk.com/gotovka_vk
https://vk.com/ynutrition
https://vk.com/club162591681
https://vk.com/ffrecept
https://vk.com/club136830288
https://vk.com/kolbasinafood
https://vk.com/club57331279
https://vk.com/techno.food
https://vk.com/receptboga
https://vk.com/vkusniashkadoma
https://vk.com/o4en_vkusno
https://vk.com/tastlife
https://vk.com/vegvkysnyashki
https://vk.com/prostogotov
https://vk.com/sekreti_krasoty
https://vk.com/koolinar_ru
https://vk.com/vkusnirecepty
https://vk.com/ladyhozyaika
https://vk.com/art.cookery
https://vk.com/public94098607
https://vk.com/xpywa
https://vk.com/public906090
https://vk.com/vk__cook
https://vk.com/povarenokru
"""
possible_recipes_publics = [-x['id'] for x in vk.groups.getById(group_ids=[b.rsplit("/")[-1] for b in a.split("\n")[1:-1]])]

In [36]:
path = pathlib.Path("/home/george/workspace/data/vegan_chef")

In [37]:
for public_id in tqdm.tqdm(possible_recipes_publics):
    file = (path/f"publics/{public_id}.json")
    if not file.exists():
        try:
            all_recipes = tools.get_all("wall.get", 80, values={"owner_id": public_id})
            json.dump(all_recipes, file.open("w"))
        except vk_api.VkToolsException:
            print(f"{public_id} public_id is broken")

 27%|██▋       | 6/22 [00:04<00:11,  1.44it/s]

-103547324 public_id is broken


 32%|███▏      | 7/22 [00:29<02:00,  8.05s/it]

-57331279 public_id is broken


 36%|███▋      | 8/22 [00:32<01:33,  6.71s/it]

...

 91%|█████████ | 20/22 [03:53<00:43, 21.54s/it]

-101431498 public_id is broken


 95%|█████████▌| 21/22 [04:28<00:25, 25.65s/it]

-49694199 public_id is broken


100%|██████████| 22/22 [05:16<00:00, 32.40s/it]


In [38]:
with open("/home/george/workspace/data/vegan_chef/all_text.txt", "w") as f:
    for public_id in tqdm.tqdm(recipes_publics + possible_recipes_publics):
        file = path/f"publics/{public_id}.json"
        if file.exists():
            all_recipes = json.load(file.open())
            for item in all_recipes['items']:
                matched = re.match("[а-яА-Я ]+", item['text'].lower())
                if matched:
                    print(matched.group(), file=f)

100%|██████████| 25/25 [00:10<00:00,  2.52it/s]


In [39]:
import sentencepiece as spm

In [40]:
spm.SentencePieceTrainer.Train(f"--input={path/'all_text.txt'} --model_prefix=m --vocab_size=1000")

True

In [41]:
!mv m.model assets/
!mv m.vocab assets/

In [42]:
sp = spm.SentencePieceProcessor()
sp.Load("assets/m.model")

True

In [43]:
tokens = [a.split("\t")[0] for a in (pathlib.Path("assets")/"m.vocab").read_text().split("\n")]

In [44]:
class PublicsIterator:
    def __init__(self, public_paths):
        self.public_paths = public_paths
    def __iter__(self):
        for public_path in self.public_paths:
            all_recipes = json.load(public_path.open())
            for recipe in all_recipes['items']:
                yield ["<s>"] + [tokens[idx] for idx in sp.EncodeAsIds(recipe['text'].lower())] + ["</s>"]

In [45]:
from gensim.models import Word2Vec

In [47]:
model = Word2Vec(
    PublicsIterator([path/f"publics/{public_id}.json" for public_id in recipes_publics]),
    size=64,
    min_count=2,
    negative=3,
    iter=10,
    workers=-1
)

In [48]:
tokens = tokens[:-1]

In [49]:
w2v_tokens = list(model.wv.vocab)

In [50]:
len(w2v_tokens)

983

In [51]:
set(tokens).difference(set(w2v_tokens))

{'▁беконом',
 '▁ветчиной',
 '▁горбуша',
 '▁грудки',
 '▁индейки',
 '▁кальмар',
 '▁креветками',
 '▁крылышки',
 '▁куриная',
 '▁куриным',
 '▁курицей',
 '▁курочка',
 '▁лосось',
 '▁отбивные',
 '▁сельдь',
 '▁скумбрия',
 '▁тунцом'}

In [52]:
w2v_tokens_set = set(w2v_tokens)

In [53]:
matched_w2v_vectors = model.wv.vectors[[w2v_tokens.index(tok) for tok in tokens if tok in w2v_tokens_set]]

In [54]:
matched_w2v_vectors[sp.EncodeAsIds("блинчики с вареньем")].mean(0)

array([ 0.00302968, -0.00091405, -0.00244807, -0.00230492, -0.00231656,
        0.00315632,  0.00073344, -0.00095295,  0.00172484,  0.00072071,
       -0.00076808,  0.00028419,  0.00337978,  0.00579838,  0.00322273,
        0.00111731,  0.00146784, -0.00183582,  0.00022545,  0.00270499,
        0.00026739,  0.00197104,  0.00139249,  0.00089627,  0.00041021,
       -0.0014714 ,  0.0032349 , -0.00246451, -0.00552011, -0.00055513,
        0.00164683, -0.00168711, -0.00219946,  0.00047899,  0.00198559,
        0.00128617,  0.00359842, -0.00459012,  0.00059059, -0.00342426,
        0.0007134 ,  0.00151454,  0.00016948,  0.00107435, -0.00083409,
       -0.00329718, -0.00070181, -0.0034659 ,  0.00169605, -0.00542269,
       -0.00046429, -0.00021056, -0.00339388,  0.00084661,  0.00435095,
       -0.00331703,  0.00125334,  0.00026163,  0.00100691,  0.00115508,
       -0.00081912,  0.00113951, -0.00161742, -0.0013855 ], dtype=float32)

In [55]:
vec1 = matched_w2v_vectors[sp.EncodeAsIds("блинчики с вареньем")].mean(0)
vec2 = matched_w2v_vectors[sp.EncodeAsIds("блины с джемом")].mean(0)
vec3 = matched_w2v_vectors[sp.EncodeAsIds("капуста с тофу")].mean(0)

In [56]:
from scipy.spatial.distance import cosine

In [57]:
1 - cosine(vec1, vec2), 1 - cosine(vec1, vec3)

(0.1698303520679474, 0.35895785689353943)

In [58]:
def compare_ingredients(ingr1, ingr2, matched_w2v_vectors, sp):
    vec1 = matched_w2v_vectors[sp.EncodeAsIds(ingr1)].mean(0)
    vec2 = matched_w2v_vectors[sp.EncodeAsIds(ingr2)].mean(0)
    return 1 - cosine(vec1, vec2)

In [59]:
import numpy as np
np.save("assets/matched_w2v_vectors.npy", matched_w2v_vectors)